In [195]:
import pandas as pd
import os
import glob
import pandas as pd 
import numpy as np
import urllib
import requests
from pathlib import Path
from zipfile import ZipFile
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px
import addfips
#import geoplot as gplt

# set options so columns aren't hidden 
pd.set_option('display.max_columns', None)

# set options so rows aren't hidden 
pd.set_option('display.max_rows', None)

In [196]:
# gdf data read in
gdf = pd.read_csv("gdf.csv", dtype='str').iloc[:, 1:]
# keeping all counties
all_counties = gdf[['GEOID','STATE_NAME','NAME']]

In [197]:
# read in noaa data with census counties
noaa_data_fin = pd.read_csv("noaa_data_fin.csv", dtype='str').iloc[:, 1:]
# set column data types
noaa_data_fin[['yr','mag']] = noaa_data_fin[['yr','mag']].apply(pd.to_numeric)

In [201]:
# aggregating counts for noaa 
# hail
agg_hail_noaa = noaa_data_fin.loc[(noaa_data_fin['type'] == 'hail') & (noaa_data_fin['mag'] >= 1) & (noaa_data_fin['yr'] >= 2018) & (noaa_data_fin['yr'] < 2023)].\
         groupby(['GEOID','STATE_NAME','NAME','type'])['om'].\
         agg({'count'}).\
         sort_values(['GEOID','STATE_NAME','NAME','type'], ascending=[True, True, True,True]).reset_index()
# modify axis
agg_hail_noaa = agg_hail_noaa.reset_index(drop = True)
# merge with counties
agg_hail_noaa = all_counties.merge(agg_hail_noaa[['GEOID','type','count']], on='GEOID', how='left')
# fill missing type
agg_hail_noaa['type'] = agg_hail_noaa['type'].fillna('hail')

# tornado
agg_tornado_noaa = noaa_data_fin.loc[(noaa_data_fin['type'] == 'tornado') & (noaa_data_fin['yr'] >= 2018) & (noaa_data_fin['yr'] < 2023)].\
         groupby(['GEOID','STATE_NAME','NAME','type'])['om'].\
         agg({'count'}).\
         sort_values(['GEOID','STATE_NAME','NAME','type'], ascending=[True, True, True,True]).reset_index()
# modify axis
agg_tornado_noaa = agg_tornado_noaa.reset_index(drop = True)
# merge with counties
agg_tornado_noaa = all_counties.merge(agg_tornado_noaa[['GEOID','type','count']], on='GEOID', how='left')
# fill missing type
agg_hail_noaa['type'] = agg_hail_noaa['type'].fillna('tornado')

# wind
agg_wind_noaa = noaa_data_fin.loc[(noaa_data_fin['type'] == 'wind') & (noaa_data_fin['yr'] >= 2018) & (noaa_data_fin['yr'] < 2023)].\
         groupby(['GEOID','STATE_NAME','NAME','type'])['om'].\
         agg({'count'}).\
         sort_values(['GEOID','STATE_NAME','NAME','type'], ascending=[True, True, True,True]).reset_index()
# modify axis
agg_wind_noaa = agg_wind_noaa.reset_index(drop = True)
# merge with counties
agg_wind_noaa = all_counties.merge(agg_wind_noaa[['GEOID','type','count']], on='GEOID', how='left')
# fill missing type
agg_hail_noaa['type'] = agg_hail_noaa['type'].fillna('wind')

# aggregation of peril types
perils = [agg_hail_noaa, agg_tornado_noaa, agg_wind_noaa]
final_noaa = pd.concat(perils)

# fix nan for counts
final_noaa['count'] = final_noaa['count'].replace(np.nan, 0)

In [206]:
len(final_noaa)

9702

In [207]:
# pop data read in
pop = pd.read_csv("pop.csv", dtype='str').iloc[:, 1:]
# adding population to noaa data
final_noaa_cnts_pop = final_noaa.merge(pop, on='GEOID', how='left')

In [208]:
len(final_noaa_cnts_pop)

9702

In [209]:
final_noaa_cnts_pop.head()

,GEOID,STATE_NAME,NAME,type,count,PopEst_2019
0,04023,Arizona,Santa Cruz,hail,11.0,46498
1,12059,Florida,Holmes,hail,0.0,19617
2,27081,Minnesota,Lincoln,hail,4.0,5639
3,30029,Montana,Flathead,hail,6.0,103806
4,31135,Nebraska,Perkins,hail,48.0,2891


In [ ]:
final_noaa_cnts_pop.to_csv("final_noaa_cnts_pop.csv", index=False)